In [23]:
from pynq import Overlay
import time
ol = Overlay('matrix.bit')

In [3]:
#ol?

In [24]:
dma = ol.axi_dma_0

In [25]:
data_send = dma.sendchannel
data_recv = dma.recvchannel

In [26]:
import numpy as np
import random 

In [27]:
random.seed(2)
data_A = np.random.uniform(low=1, high=2, size=(8, 8)).astype(float)
data_B = np.random.uniform(low=1, high=2, size=(8, 8)).astype(float)

In [28]:
data_A, data_B

(array([[1.12728203, 1.15096545, 1.30285636, 1.52102975, 1.59551539,
         1.12167806, 1.97479664, 1.41991747],
        [1.877379  , 1.45326106, 1.29213282, 1.61802064, 1.02670174,
         1.05145811, 1.94006155, 1.65443863],
        [1.04297803, 1.84546127, 1.8065279 , 1.05566474, 1.77629876,
         1.32271852, 1.6608249 , 1.71436733],
        [1.02827022, 1.76695882, 1.35664304, 1.40486903, 1.17572893,
         1.29928667, 1.93225632, 1.20677264],
        [1.66767802, 1.35421569, 1.18112096, 1.79928481, 1.06754512,
         1.82125496, 1.79798671, 1.27322278],
        [1.20689038, 1.18710047, 1.11735703, 1.25442044, 1.92152981,
         1.78692859, 1.02017292, 1.24884923],
        [1.25340225, 1.05534272, 1.0579971 , 1.37115047, 1.37536538,
         1.72806876, 1.51341755, 1.19553631],
        [1.16079011, 1.19531749, 1.43901365, 1.05352565, 1.14250403,
         1.48128002, 1.60694076, 1.07961916]]),
 array([[1.94429503, 1.43163899, 1.30082574, 1.71887852, 1.64781499,
         

In [29]:
def multiply(matrix_a, matrix_b):
    result_matrix = np.zeros((8,8))
    for i in range(8):
        for j in range(8):
            for k in range(8):
                result_matrix[i][j] += matrix_a[i][k] * matrix_b[k][j]
                
    return result_matrix

In [30]:
start_time = time.time()
multiply(data_A, data_B) #software
end_time = time.time()
print(end_time - start_time)

0.0038785934448242188


In [31]:
from pynq import allocate

In [32]:
data_a_f = data_A.flatten()
data_b_f = data_B.flatten()

data_input = np.concatenate([data_a_f, data_b_f])

In [33]:
data_input

array([1.12728203, 1.15096545, 1.30285636, 1.52102975, 1.59551539,
       1.12167806, 1.97479664, 1.41991747, 1.877379  , 1.45326106,
       1.29213282, 1.61802064, 1.02670174, 1.05145811, 1.94006155,
       1.65443863, 1.04297803, 1.84546127, 1.8065279 , 1.05566474,
       1.77629876, 1.32271852, 1.6608249 , 1.71436733, 1.02827022,
       1.76695882, 1.35664304, 1.40486903, 1.17572893, 1.29928667,
       1.93225632, 1.20677264, 1.66767802, 1.35421569, 1.18112096,
       1.79928481, 1.06754512, 1.82125496, 1.79798671, 1.27322278,
       1.20689038, 1.18710047, 1.11735703, 1.25442044, 1.92152981,
       1.78692859, 1.02017292, 1.24884923, 1.25340225, 1.05534272,
       1.0579971 , 1.37115047, 1.37536538, 1.72806876, 1.51341755,
       1.19553631, 1.16079011, 1.19531749, 1.43901365, 1.05352565,
       1.14250403, 1.48128002, 1.60694076, 1.07961916, 1.94429503,
       1.43163899, 1.30082574, 1.71887852, 1.64781499, 1.25332341,
       1.65048924, 1.79515184, 1.75598692, 1.22936872, 1.33371

In [34]:
input_buffer = allocate(128, np.float32)
output_buffer = allocate((8,8), np.float32)

In [35]:
np.copyto(input_buffer, data_input)

In [37]:
start_time = time.time()
data_send.transfer(input_buffer)
data_recv.transfer(output_buffer)
data_send.wait()
data_recv.wait()
end_time = time.time()
print(end_time - start_time)

0.0016579627990722656


In [38]:
output_buffer

PynqBuffer([[18.661919, 18.198532, 16.021223, 16.681107, 15.890083,
             16.345778, 18.974041, 16.797108],
            [19.933609, 18.93319 , 16.72073 , 18.073582, 17.082047,
             17.23758 , 19.955484, 18.09167 ],
            [20.151093, 19.253128, 17.396458, 18.444746, 17.368637,
             17.998245, 20.780321, 18.334776],
            [18.725088, 17.795837, 15.695977, 16.757874, 16.130674,
             15.978685, 19.131104, 16.959887],
            [20.354237, 19.325352, 16.646809, 18.050327, 16.964066,
             16.98275 , 20.379574, 18.00145 ],
            [18.159985, 17.436186, 15.317394, 16.025415, 15.012647,
             15.504355, 18.42616 , 15.749697],
            [17.808683, 17.167473, 14.914955, 15.862132, 14.84883 ,
             15.092407, 18.009556, 15.678173],
            [17.062637, 16.200865, 14.22896 , 15.417045, 14.401366,
             14.655547, 17.395908, 15.425227]], dtype=float32)

In [39]:
diff = np.dot(data_A, data_B) - output_buffer

In [40]:
rms_error = np.sqrt(np.mean(diff**2))

In [41]:
diff, rms_error

(PynqBuffer([[ 7.23172629e-07,  1.25105328e-06,  5.82738920e-07,
               4.97047377e-07,  1.42031845e-06, -8.60996316e-07,
              -1.13356936e-07, -5.07033938e-07],
             [ 7.14933179e-07,  1.23552125e-06,  1.42715601e-06,
              -8.00653144e-07, -5.01467525e-07,  4.14920429e-07,
              -3.09621932e-07, -9.52745438e-09],
             [ 1.81085689e-08,  1.25181553e-06,  6.00309008e-07,
              -8.00971545e-07,  3.94249504e-07,  3.46589445e-07,
              -3.47010197e-07,  2.69607025e-07],
             [-3.80317392e-07,  8.64177544e-07,  1.56195252e-07,
              -1.97036558e-06,  2.45735674e-07,  1.23197131e-07,
               9.40688345e-07,  1.54563278e-07],
             [-2.09539788e-06,  1.63290187e-06, -1.50894667e-06,
              -1.53443738e-06,  4.42608236e-07, -8.49852398e-07,
              -6.07164825e-07, -1.43007036e-06],
             [ 3.43139465e-07,  7.36493774e-07,  1.18100805e-06,
               2.39961221e-07,  6.888656

In [42]:
%%time
np.dot(data_A, data_B)

CPU times: user 393 µs, sys: 78 µs, total: 471 µs
Wall time: 437 µs


array([[18.66191936, 18.19853336, 16.02122365, 16.68110706, 15.89008473,
        16.34577665, 18.97404087, 16.79710719],
       [19.93360972, 18.93319063, 16.72073126, 18.07358089, 17.08204601,
        17.23757976, 19.95548408, 18.09166907],
       [20.15109255, 19.2531293 , 17.39645827, 18.44474522, 17.36863748,
        17.99824559, 20.78032077, 18.33477619],
       [18.72508774, 17.79583827, 15.69597737, 16.75787156, 16.13067461,
        15.9786855 , 19.13110446, 16.95988671],
       [20.35423451, 19.32535335, 16.64680712, 18.05032577, 16.96406599,
        16.98274909, 20.37957321, 18.00144815],
       [18.15998493, 17.43618657, 15.31739544, 16.02541566, 15.01264736,
        15.50435648, 18.42616158, 15.74969784],
       [17.80868255, 17.16747258, 14.91495456, 15.86213226, 14.84883053,
        15.09240682, 18.00955542, 15.67817386],
       [17.0626365 , 16.20086501, 14.228961  , 15.41704514, 14.40136613,
        14.65554758, 17.39590716, 15.4252276 ]])

In [43]:
rms_error

PynqBuffer(8.49777596e-07)